In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from scipy.stats import levene, ttest_ind, pearsonr
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, plot_confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_text
import math
from scipy import stats
import numpy as np
import re
import os
import pandas as pd
import requests
import seaborn as sns
import statsmodels.api as sm
import warnings
import wrangle as w
import re
warnings.filterwarnings("ignore")

In [2]:
sns.set(rc={'figure.figsize':(16,9)})

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
df=w.get_explore_data()

In [5]:
df=w.get_contest_data(df)

In [6]:
df.head(50)

,region,elevation,lat,lon,startdate,potential_evap,precip,barometric_pressure,all_atmos_precip,relative_humidity,sea_level_press,mean_temp,height_10_mb,height_100_mb,height_500_mb,height_850_mb,zonal_wind_250mb,zonal_wind_925mb,long_wind_250mb,long_wind_925mb,elevation_range,region_bins
0,BSh,200,0.0,0.833333,2014-09-01,237.00,94.31,98644.97,42.45,81.72,101352.08,28.744480,31246.63,16666.81,5899.66,1535.52,-2.56,-5.22,-3.52,4.41,bottom_low,Dry
1,BSh,200,0.0,0.833333,2014-09-02,228.90,100.85,98686.80,42.66,82.56,101396.02,28.370585,31244.78,16667.31,5901.03,1538.00,-2.39,-5.20,-4.49,3.74,bottom_low,Dry
2,BSh,200,0.0,0.833333,2014-09-03,220.69,101.25,98712.85,43.23,83.29,101429.25,28.133059,31239.27,16668.39,5902.18,1540.32,-2.76,-5.00,-5.44,3.40,bottom_low,Dry
3,BSh,200,0.0,0.833333,2014-09-04,225.28,101.90,98711.70,43.11,83.26,101440.85,28.256798,31232.86,16667.39,5903.07,1541.10,-3.00,-4.61,-5.76,3.29,bottom_low,Dry
4,BSh,200,0.0,0.833333,2014-09-05,237.24,82.95,98686.46,42.98,82.50,101419.53,28.372353,31226.16,16665.65,5903.36,1539.73,-3.40,-4.25,-6.09,3.27,bottom_low,Dry
5,BSh,200,0.0,0.833333,2014-09-06,237.87,79.11,98649.75,42.98,82.46,101380.35,28.523149,31222.25,16664.20,5901.94,1536.71,-3.71,-4.00,-6.02,3.14,bottom_low,Dry
6,BSh,200,0.0,0.833333,2014-09-07,236.36,81.54,98619.14,43.28,82.68,101349.89,28.451247,31216.90,16665.01,5900.75,1534.26,-3.47,-3.99,-5.38,2.92,bottom_low,Dry
7,BSh,200,0.0,0.833333,2014-09-08,233.36,87.87,98609.42,44.12,83.11,101335.59,28.313029,31210.24,16667.35,5901.17,1533.13,-3.24,-4.02,-4.99,2.71,bottom_low,Dry
8,BSh,200,0.0,0.833333,2014-09-09,233.82,96.14,98625.38,44.46,83.35,101342.69,28.123519,31200.53,16668.39,5902.21,1533.40,-2.73,-4.26,-5.51,2.38,bottom_low,Dry
9,BSh,200,0.0,0.833333,2014-09-10,229.74,96.13,98651.15,44.37,83.54,101365.05,27.885300,31188.35,16665.66,5901.96,1534.67,-1.83,-4.38,-6.40,1.84,bottom_low,Dry


In [7]:
def split_data(df, test_size=0.15):
    '''
    Takes in a data frame and the train size
    It returns train, validate , and test data frames
    with validate being 0.05 bigger than test and train has the rest of the data.
    '''
    train, test = train_test_split(df, test_size = test_size , random_state=27)
    train, validate = train_test_split(train, test_size = (test_size + 0.05)/(1-test_size), random_state=27)
    
    return train, validate, test

In [8]:
train, validate, test =split_data(df, test_size=0.15)

In [9]:
train.head()

,region,elevation,lat,lon,startdate,potential_evap,precip,barometric_pressure,all_atmos_precip,relative_humidity,sea_level_press,mean_temp,height_10_mb,height_100_mb,height_500_mb,height_850_mb,zonal_wind_250mb,zonal_wind_925mb,long_wind_250mb,long_wind_925mb,elevation_range,region_bins
116277,BSk,1800,0.454545,0.300000,2014-10-19,294.86,0.03,83362.52,8.53,28.94,101443.38,16.680867,30825.95,16475.47,5796.30,1504.26,18.22,-0.87,3.46,2.04,high,Dry
237629,Csb,1100,0.727273,0.066667,2014-10-25,127.70,63.74,89156.29,16.71,83.14,101841.63,5.987106,30801.60,16340.47,5689.14,1504.33,24.22,1.52,1.08,3.10,mid,Temperate
272599,BSk,800,0.772727,0.700000,2016-06-29,481.48,41.43,89665.38,22.36,54.72,101003.07,21.549640,31605.85,16624.04,5828.97,1492.70,23.68,-0.08,2.35,1.04,top_low,Dry
205713,Dfb,2000,0.636364,0.500000,2015-06-30,453.35,24.78,77957.43,17.11,47.80,101361.57,19.173942,31576.02,16662.35,5875.87,1505.58,15.35,0.50,5.52,1.09,high,Continental
120041,BSk,1900,0.454545,0.466667,2015-02-05,135.47,0.00,81613.28,6.06,62.50,102053.60,6.862285,30715.39,16357.43,5755.62,1535.91,19.55,0.42,-10.67,0.27,high,Dry


# Getting mean_temp per unique lat and lon by region

Here is a step-by-step explanation of this code:

df_BSh = df[df['region'] == 'BSh']: This line filters the dataframe to only include the rows where the 'region' column has the value 'BSh'.

grouped = df_BSh.groupby(['lat', 'lon'], as_index=False): This line groups the filtered dataframe by the unique values of the 'lat' and 'lon' columns. The as_index=False argument ensures that the resulting groups are not used as the index of the grouped dataframe.

mean_temps = []: This line creates an empty list that will be used to store the mean of the 'mean_temp' column for each group.

for name, group in grouped:: This line starts a for loop that iterates through the groups generated in step 2. For each iteration, name is a tuple representing the values of 'lat' and 'lon' for the current group, and group is a dataframe representing the rows in df_BSh that belong to the current group.

mean_temp = group['mean_temp'].mean(): This line calculates the mean of the 'mean_temp' column for the current group.

mean_temps.append(mean_temp): This line adds the mean from step 5 to the mean_temps list.

`mean_temp_mean = sum(mean_temps)/len(mean_temps)

In [10]:
#region BSh
df_BSh = train[train['region'] == 'BSh']
grouped = df_BSh.groupby(['lat', 'lon'], as_index=False)
mean_temps = []
for name, group in grouped:
    mean_temp = group['mean_temp'].mean()
    mean_temps.append(mean_temp)

mean_temp_mean = sum(mean_temps)/len(mean_temps)

# the average for this region 
mean_temp_mean

# the average for unique combos of at and lon in that region
mean_temps

[23.274079372633,
 20.57022143744541,
 20.562598291862134,
 19.097334528571427,
 19.384368690032623,
 23.074148924504282,
 19.90265784524737]

In [11]:
#region BSk
df_BSk = train[train['region'] == 'BSk']
grouped = df_BSk.groupby(['lat', 'lon'], as_index=False)
mean_temps = []
for name, group in grouped:
    mean_temp = group['mean_temp'].mean()
    mean_temps.append(mean_temp)

mean_temp_mean = sum(mean_temps)/len(mean_temps)

# the average for this region 
mean_temp_mean

# the average for unique combos of at and lon in that region
mean_temps

[16.88129937208894,
 18.267491460144353,
 18.761438535530658,
 16.6713670526245,
 16.251226194454752,
 16.53420323982009,
 18.6418384650253,
 18.734759337156287,
 18.43215572974242,
 16.733494858102507,
 12.104758873656461,
 14.984315071032787,
 16.372694936197547,
 16.99090191191188,
 17.557819425384935,
 17.651061911667355,
 20.173177107006435,
 9.365488510410872,
 10.269400671162845,
 13.94727256350001,
 12.851557975821429,
 13.792448133728408,
 15.501869116308475,
 15.802579858311677,
 15.882254682466248,
 16.62714866306513,
 17.044997234232607,
 15.495972224355173,
 16.616737457583515,
 19.006109172734906,
 14.42957177645568,
 12.052208521745829,
 12.666564884756465,
 11.26474445742251,
 9.56461570088889,
 13.619188131859968,
 12.252133890943387,
 13.062565914523597,
 14.277500231204122,
 15.079336754989376,
 14.793076006572965,
 16.02210009112064,
 17.474038764571137,
 13.84466437181877,
 17.573137553348023,
 13.513061882233265,
 11.589746087472527,
 13.173499885113685,
 10.95109

In [12]:
# region BWh
df_BWh = train[train['region'] == 'BWh']
grouped = df_BWh.groupby(['lat', 'lon'], as_index=False)
mean_temps = []
for name, group in grouped:
    mean_temp = group['mean_temp'].mean()
    mean_temps.append(mean_temp)

mean_temp_mean = sum(mean_temps)/len(mean_temps)

# the average for this region 
mean_temp_mean

# the average for unique combos of at and lon in that region
mean_temps

[16.580615000310488,
 19.011646551705756,
 24.76306292008601,
 21.833065109482693,
 24.72341727593939,
 24.585600268646807,
 25.016378134404423,
 23.673207858414646,
 23.5449830300165,
 22.951940477189037,
 21.132944609613375,
 20.850159873302427,
 19.535371271653776]

In [13]:
# region BWk 
df_BWk = train[train['region'] == 'BWk']
grouped = df_BWk.groupby(['lat', 'lon'], as_index=False)
mean_temps = []
for name, group in grouped:
    mean_temp = group['mean_temp'].mean()
    mean_temps.append(mean_temp)

mean_temp_mean = sum(mean_temps)/len(mean_temps)

# the average for this region 
mean_temp_mean

# the average for unique combos of at and lon in that region
mean_temps

[17.314826683930605,
 17.294600380525317,
 17.0636548509833,
 17.538366996237794,
 15.855980507630797,
 20.94252223641485,
 17.08768045041528,
 18.725890345401773,
 19.576159109646046,
 21.08596596015288,
 13.151852437293751,
 17.138661015838633,
 15.959834154799163,
 12.389791644616313,
 12.589145378632479,
 13.38159966806143,
 13.101089485679232,
 11.056627549172132,
 9.07655500641478]

In [14]:
# region Cfa
df_Cfa = train[train['region'] == 'Cfa']
grouped = df_Cfa.groupby(['lat', 'lon'], as_index=False)
mean_temps = []
for name, group in grouped:
    mean_temp = group['mean_temp'].mean()
    mean_temps.append(mean_temp)

mean_temp_mean = sum(mean_temps)/len(mean_temps)

# the average for this region 
mean_temp_mean

# the average for unique combos of at and lon in that region
mean_temps

[23.188255424910054,
 23.345404557804557,
 22.609616018023555,
 22.19631665290702,
 22.94123926876571,
 22.465552967442665,
 21.73791237457271,
 21.52641685776972,
 21.742060527934974,
 20.535597926148792,
 20.100786999928424,
 20.11332475178926,
 20.670133907486196,
 21.10581552792662,
 21.416281770492876,
 21.205891712046164,
 19.133693903834427,
 19.137910739035405,
 19.645175062081265,
 20.24760714841249,
 20.39824802048147,
 20.041989866694323,
 19.98463978296024,
 18.824624569750014,
 18.76829010297858,
 19.34729044302597,
 19.358425455312123,
 19.638151076646214,
 19.35105286693103,
 18.330551763546207,
 18.214012123944016,
 19.460989293995656,
 19.541982000210314,
 18.82507141532097,
 19.03446464615284,
 17.7119841945133,
 17.65773247136653,
 17.451531728551558,
 17.695155210656527,
 17.79635854652857,
 16.23120804489959,
 16.29799874017464,
 16.820007352454912,
 16.755337741766805,
 17.090010377247395,
 17.448741474028566,
 15.895533148349035,
 16.05340936784849,
 17.038404147

In [15]:
# region Cfb
df_Cfb = train[train['region'] == 'Cfb']
grouped = df_Cfb.groupby(['lat', 'lon'], as_index=False)
mean_temps = []
for name, group in grouped:
    mean_temp = group['mean_temp'].mean()
    mean_temps.append(mean_temp)

mean_temp_mean = sum(mean_temps)/len(mean_temps)

# the average for this region 
mean_temp_mean

# the average for unique combos of at and lon in that region
mean_temps

[11.532037533943834,
 10.209679076137135,
 9.955810924964592,
 11.978660651637739,
 11.324809793790491,
 10.338300022046846]

In [16]:
# region Csa
df_Csa = train[train['region'] == 'Csa']
grouped = df_Csa.groupby(['lat', 'lon'], as_index=False)
mean_temps = []
for name, group in grouped:
    mean_temp = group['mean_temp'].mean()
    mean_temps.append(mean_temp)

mean_temp_mean = sum(mean_temps)/len(mean_temps)

# the average for this region 
mean_temp_mean

# the average for unique combos of at and lon in that region
mean_temps

[17.775937622724282,
 21.07584791201924,
 19.33228408009522,
 15.32769547510892,
 18.20154554160332,
 16.22890700320382,
 17.28355426627401,
 16.98044400521383,
 12.358135573921945,
 12.503545650139229,
 11.894876565522441,
 11.643503082779684,
 12.913108130597914]

In [17]:
# region Csb
df_Csb = train[train['region'] == 'Csb']
grouped = df_Csb.groupby(['lat', 'lon'], as_index=False)
mean_temps = []
for name, group in grouped:
    mean_temp = group['mean_temp'].mean()
    mean_temps.append(mean_temp)

mean_temp_mean = sum(mean_temps)/len(mean_temps)

# the average for this region 
mean_temp_mean

# the average for unique combos of at and lon in that region
mean_temps

[13.762092284006519,
 14.220950733628634,
 18.26982775333335,
 16.682121011904545,
 11.974696947884862,
 15.242564254877594,
 17.515965530089947,
 15.389701514479672,
 13.214855957918575,
 12.387247894045549,
 9.267423246083146,
 14.34603153485131,
 15.948318797335363,
 10.67169961642651,
 11.332917096184422,
 16.528354374715754,
 11.25711807944577,
 11.222450226172562,
 10.26693465581653,
 12.46565387847176,
 10.62212853763002,
 10.169665133748955,
 11.500272175228998,
 11.980687754803,
 10.103563733321797,
 9.142237954689518,
 8.97883955257452,
 12.694990582957317,
 12.581732849192308,
 8.058986878401278,
 7.0292749215952925,
 12.506281630117408,
 12.236863015336903,
 8.459246163693473,
 8.241870976534045,
 7.797769462314894,
 12.487503223768247,
 13.6831681821928,
 10.49841845583879,
 11.193176553451936,
 9.755512972742622,
 8.216121972533188,
 7.391586473676147,
 7.354976550037111,
 16.101300439603456,
 11.952877277703003,
 11.181276707220752,
 9.904875041921159,
 9.36440877567261,

In [18]:
# region Dfa
df_Dfa = train[train['region'] == 'Dfa']
grouped = df_Dfa.groupby(['lat', 'lon'], as_index=False)
mean_temps = []
for name, group in grouped:
    mean_temp = group['mean_temp'].mean()
    mean_temps.append(mean_temp)

mean_temp_mean = sum(mean_temps)/len(mean_temps)

# the average for this region 
mean_temp_mean

# the average for unique combos of at and lon in that region
mean_temps

[10.720203442383026,
 11.936884302553713,
 12.321940019338463,
 12.183890980149364,
 12.258861742262594,
 12.503936300181241,
 12.75774662635204,
 10.685652348271999,
 10.826341977801306,
 11.024053620776597,
 11.607564140217653,
 11.79744318498745,
 11.929454150909475,
 9.509291103819917,
 10.01635787780321,
 10.101419126411258,
 10.01581392612897,
 10.278712565911253,
 10.767278502808514,
 9.946842158357152,
 10.494259352600002,
 9.94601435292291,
 10.206602060187885,
 9.548257703078221,
 8.86352718034426,
 8.990869176995721,
 8.88921277055602,
 9.070492849939953,
 8.839954877274518,
 9.402451318152746,
 8.004889445969432]

In [19]:
# region Dfb
df_Dfb = train[train['region'] == 'Dfb']
grouped = df_BSh.groupby(['lat', 'lon'], as_index=False)
mean_temps = []
for name, group in grouped:
    mean_temp = group['mean_temp'].mean()
    mean_temps.append(mean_temp)

mean_temp_mean = sum(mean_temps)/len(mean_temps)

# the average for this region 
mean_temp_mean

# the average for unique combos of at and lon in that region
mean_temps

[23.274079372633,
 20.57022143744541,
 20.562598291862134,
 19.097334528571427,
 19.384368690032623,
 23.074148924504282,
 19.90265784524737]

In [20]:
# region Dfc
df_Dfc = train[train['region'] == 'Dfc']
grouped = df_BSh.groupby(['lat', 'lon'], as_index=False)
mean_temps = []
for name, group in grouped:
    mean_temp = group['mean_temp'].mean()
    mean_temps.append(mean_temp)

mean_temp_mean = sum(mean_temps)/len(mean_temps)

# the average for this region 
mean_temp_mean

# the average for unique combos of at and lon in that region
mean_temps

[23.274079372633,
 20.57022143744541,
 20.562598291862134,
 19.097334528571427,
 19.384368690032623,
 23.074148924504282,
 19.90265784524737]

In [21]:
# region Dsb
df_Dsb = train[train['region'] == 'Dsb']
grouped = df_Dsb.groupby(['lat', 'lon'], as_index=False)
mean_temps = []
for name, group in grouped:
    mean_temp = group['mean_temp'].mean()
    mean_temps.append(mean_temp)

mean_temp_mean = sum(mean_temps)/len(mean_temps)

# the average for this region 
mean_temp_mean

# the average for unique combos of at and lon in that region
mean_temps

[10.319294517223371,
 9.579931324585207,
 6.789482745827373,
 8.0837581947914,
 6.40120783885042,
 8.991636946578838,
 9.416640963892485,
 10.363242666317898,
 10.556227199451296,
 9.733795643290527,
 8.403377506617202]

In [22]:
# region Dsc
df_Dsc = train[train['region'] == 'Dsc']
grouped = df_Dsc.groupby(['lat', 'lon'], as_index=False)
mean_temps = []
for name, group in grouped:
    mean_temp = group['mean_temp'].mean()
    mean_temps.append(mean_temp)

mean_temp_mean = sum(mean_temps)/len(mean_temps)

# the average for this region 
mean_temp_mean

# the average for unique combos of at and lon in that region
mean_temps

[11.64062151601853, 5.880885536063556, 6.456988234825552, 5.6085412991353065]

In [23]:
# region Dwa
df_Dwa = train[train['region'] == 'Dwa']
grouped = df_BSh.groupby(['lat', 'lon'], as_index=False)
mean_temps = []
for name, group in grouped:
    mean_temp = group['mean_temp'].mean()
    mean_temps.append(mean_temp)

mean_temp_mean = sum(mean_temps)/len(mean_temps)

# the average for this region 
mean_temp_mean

# the average for unique combos of at and lon in that region
mean_temps

[23.274079372633,
 20.57022143744541,
 20.562598291862134,
 19.097334528571427,
 19.384368690032623,
 23.074148924504282,
 19.90265784524737]

In [24]:
# region Dwb
df_Dwb = train[train['region'] == 'Dwb']
grouped = df_Dwb.groupby(['lat', 'lon'], as_index=False)
mean_temps = []
for name, group in grouped:
    mean_temp = group['mean_temp'].mean()
    mean_temps.append(mean_temp)

mean_temp_mean = sum(mean_temps)/len(mean_temps)

# the average for this region 
mean_temp_mean

# the average for unique combos of at and lon in that region
mean_temps

[8.243420470760343]